In [1]:
import json # Импортируем модуль json
from pprint import pprint # Импортируем функцию pprint()
import pandas as pd # Импортируем модуль pandas
import re

with open("data.json", encoding='utf-8', newline='') as f:  # Открываем файл и связываем его с объектом "f"
    news = json.load(f)  # Загружаем содержимое открытого файла в переменную news
    
#pprint(news) # Выводим на экран содержимое переменной recipes, используя функцию pprint()

In [2]:
with open("data.json", encoding='utf-8', newline='') as f:  # Открываем файл и связываем его с объектом "f"
    news_list = json.load(f)   
n = 0    
for i in news_list:
    n +=1
print('Количество новостей в базе:', n)

Количество новостей в базе: 190


In [3]:
with open("data.json", encoding='utf-8', newline='') as f:  # Открываем файл и связываем его с объектом "f"
    news = json.load(f) 
data = pd.DataFrame(news_list) # Создаём объект DataFrame из списка news
df = data.copy()
df = df.T.reset_index()
display(df.head()) # Выводим на экран первые строки полученного DataFrame

,index,time_post,title_post,link,contents
0,155283,07.10.2022,"Повышение лимитов для УСН, упрощенные правила ...",https://buh.ru/news/uchet_nalogi/155283/,Редакция БУХ.1С напоминает о самых позитивных ...
1,153831,07.10.2022,Единый семинар 1С для бухгалтеров и руководите...,https://buh.ru/news/uchet_nalogi/153831/,12 октября 2022 в 10:00 по местному времени фи...
2,155279,07.10.2022,Работодателей обязали сдавать СЗВ-ТД при приос...,https://buh.ru/news/uchet_nalogi/155279/,С 7 октября 2022 года вступили в силу поправки...
3,155278,07.10.2022,Выпущена версия 3.0.222.1 конфигурации «Налого...,https://buh.ru/news/automation/155278/,"Выпущена версия 3.0.222.1 конфигурации ""Налого..."
4,155277,07.10.2022,Вправе ли работодатель отправлять форму СТД-Р ...,https://buh.ru/news/uchet_nalogi/155277/,"Эксперты Роструда разъяснили, вправе ли работо..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       190 non-null    object
 1   time_post   190 non-null    object
 2   title_post  190 non-null    object
 3   link        190 non-null    object
 4   contents    190 non-null    object
dtypes: object(5)
memory usage: 7.5+ KB


In [5]:
# Убираем рекламные новости с сайта
df = df[~df['title_post'].str.contains('1С')]
df = df[~df['contents'].str.contains('1С')]
df = df[~df['title_post'].str.contains('Выпущена версия')]
df.head()

,index,time_post,title_post,link,contents
2,155279,07.10.2022,Работодателей обязали сдавать СЗВ-ТД при приос...,https://buh.ru/news/uchet_nalogi/155279/,С 7 октября 2022 года вступили в силу поправки...
4,155277,07.10.2022,Вправе ли работодатель отправлять форму СТД-Р ...,https://buh.ru/news/uchet_nalogi/155277/,"Эксперты Роструда разъяснили, вправе ли работо..."
7,155274,07.10.2022,ФНС установит для перевозчиков формат электрон...,https://buh.ru/news/uchet_nalogi/155274/,ФНС РФ разработала формат электронного договор...
10,155271,07.10.2022,В ТК РФ внесли дополнительные трудовые гаранти...,https://buh.ru/news/uchet_nalogi/155271/,"В Трудовой кодекс внесены новые положения, кот..."
11,155266,07.10.2022,Вступил в силу закон о кредитных каникулах для...,https://buh.ru/news/uchet_nalogi/155266/,С 7 октября 2022 года вступил в силу Федеральн...


In [6]:
# Подгружаем ключевые слова для роли Бухгалтер
top_buh = pd.read_excel('Words.xlsx')
top_buh = top_buh['Ключевые слова'].tolist()

In [7]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import nltk
from nltk.stem import SnowballStemmer
snowball = SnowballStemmer(language="russian")

In [8]:
# Преобразуем текст в список слов, убирая знаки препинания, а также окончания
def getwords(text):
    words = []
    for split_str in text.split():
        split_str = morph.parse(split_str)[0].normal_form
        split_str = snowball.stem(split_str)
        t = re.split("[\s;:\-_*\".,?!()'&#«»]", split_str)     
        words += [a for a in t if a != '' and len(a) > 2]
    return words

In [9]:
# Создаем столбец со списком слов:
df['words'] = df['contents'].apply(getwords)
df.head()

,index,time_post,title_post,link,contents,words
2,155279,07.10.2022,Работодателей обязали сдавать СЗВ-ТД при приос...,https://buh.ru/news/uchet_nalogi/155279/,С 7 октября 2022 года вступили в силу поправки...,"[октябр, 2022, год, вступ, сил, поправк, закон..."
4,155277,07.10.2022,Вправе ли работодатель отправлять форму СТД-Р ...,https://buh.ru/news/uchet_nalogi/155277/,"Эксперты Роструда разъяснили, вправе ли работо...","[эксперт, роструд, разъяснили, вправ, работода..."
7,155274,07.10.2022,ФНС установит для перевозчиков формат электрон...,https://buh.ru/news/uchet_nalogi/155274/,ФНС РФ разработала формат электронного договор...,"[фнс, разработа, формат, электрон, договор, фр..."
10,155271,07.10.2022,В ТК РФ внесли дополнительные трудовые гаранти...,https://buh.ru/news/uchet_nalogi/155271/,"В Трудовой кодекс внесены новые положения, кот...","[трудов, кодекс, внест, нов, положения, котор,..."
11,155266,07.10.2022,Вступил в силу закон о кредитных каникулах для...,https://buh.ru/news/uchet_nalogi/155266/,С 7 октября 2022 года вступил в силу Федеральн...,"[октябр, 2022, год, вступ, сил, федеральн, зак..."


In [10]:
# Определяем ключевые слова для роли, которые присутствуют в тексте новости
def tops(text): # Определяем имя функции и передаваемые аргументы
    n = []
    for word in top_buh:
        if word.lower() in text: 
            n.append(word)
    return n # возвращаем слово

In [11]:
# Преобразуем столбец к datetime
df['time_post'] =  pd.to_datetime(df['time_post'], format="%d.%m.%Y", dayfirst=True);

In [12]:
# Добавляем столбцы с ключевыми словами и количеством ключевых слов в тексте
#n = []for word in top: # Последовательно перебираем 
df['top'] = df['words'].apply(tops) # В DataFrame cоздаём столбец 
df["count"] = df["top"].apply(lambda x: len(x))

df = df.sort_values('count', ascending=False)
#df

In [13]:
df['top_title'] = df['title_post'].apply(tops) # В DataFrame cоздаём столбец 
df["count_title"] = df["top_title"].apply(lambda x: len(x))

In [14]:
stop = pd.read_excel('Stop_words.xlsx')
stop = stop['Ключевые слова'].tolist()

In [15]:
def stop_func(text): # Определяем имя функции и передаваемые аргументы
    n = []
    for word in stop:
        if word.lower() in text: 
            n.append(word)
    return n # возвращаем слово

In [16]:
df['stop'] = df['title_post'].apply(stop_func) # В DataFrame cоздаём столбец 
df["count_stop"] = df["stop"].apply(lambda x: len(x))

df = df.sort_values('count', ascending=False)
df.head()

,index,time_post,title_post,link,contents,words,top,count,top_title,count_title,stop,count_stop
85,154956,2022-09-30,ФНС завершит в октябре рассылку налоговых увед...,https://buh.ru/news/uchet_nalogi/154956/,"ФНС предупредила налогоплательщиков, что во вс...","[фнс, предупред, налогоплательщиков, что, реги...","[Изменения, Налог, ФНС]",3,[Налог],1,[],0
150,154516,2022-09-23,Можно ли сдать 4-ФСС в электронной форме без ЭЦП,https://buh.ru/news/uchet_nalogi/154516/,"ФСС разъяснил, может ли работодатель сдать в э...","[фсс, разъяснил, моч, работодател, сдат, элект...","[Отчет, Организация]",2,[],0,[],0
187,154397,2022-09-20,Банковские гарантии для возмещения НДС и акциз...,https://buh.ru/news/uchet_nalogi/154397/,Минфин предлагает закрепить в НК РФ применение...,"[минфин, предлага, закреп, применен, налогопла...","[ФНС, НДС]",2,[],0,[],0
14,155259,2022-10-07,Как мобилизованному сохранить статус ИП и прио...,https://buh.ru/news/uchet_nalogi/155259/,"Эксперты ФНС разъяснили, как мобилизованный ИП...","[эксперт, фнс, разъяснили, как, мобилизова, пр...","[Налог, ФНС]",2,[Налог],1,[],0
20,155246,2022-10-06,Можно ли списать расходы командированного на п...,https://buh.ru/news/uchet_nalogi/155246/,"Минфин разъяснил, как в целях налога на прибыл...","[минфин, разъяснил, как, цел, налог, прибыл, у...","[Налог, Утвержден]",2,[],0,[],0


In [17]:
df.head()

,index,time_post,title_post,link,contents,words,top,count,top_title,count_title,stop,count_stop
85,154956,2022-09-30,ФНС завершит в октябре рассылку налоговых увед...,https://buh.ru/news/uchet_nalogi/154956/,"ФНС предупредила налогоплательщиков, что во вс...","[фнс, предупред, налогоплательщиков, что, реги...","[Изменения, Налог, ФНС]",3,[Налог],1,[],0
150,154516,2022-09-23,Можно ли сдать 4-ФСС в электронной форме без ЭЦП,https://buh.ru/news/uchet_nalogi/154516/,"ФСС разъяснил, может ли работодатель сдать в э...","[фсс, разъяснил, моч, работодател, сдат, элект...","[Отчет, Организация]",2,[],0,[],0
187,154397,2022-09-20,Банковские гарантии для возмещения НДС и акциз...,https://buh.ru/news/uchet_nalogi/154397/,Минфин предлагает закрепить в НК РФ применение...,"[минфин, предлага, закреп, применен, налогопла...","[ФНС, НДС]",2,[],0,[],0
14,155259,2022-10-07,Как мобилизованному сохранить статус ИП и прио...,https://buh.ru/news/uchet_nalogi/155259/,"Эксперты ФНС разъяснили, как мобилизованный ИП...","[эксперт, фнс, разъяснили, как, мобилизова, пр...","[Налог, ФНС]",2,[Налог],1,[],0
20,155246,2022-10-06,Можно ли списать расходы командированного на п...,https://buh.ru/news/uchet_nalogi/155246/,"Минфин разъяснил, как в целях налога на прибыл...","[минфин, разъяснил, как, цел, налог, прибыл, у...","[Налог, Утвержден]",2,[],0,[],0


In [18]:
from datetime import datetime, timedelta
yesterday = datetime.now() - timedelta(days=1)

yesterday.date() # не помню как привести дату к тому же виду как в таблице!!!!!

datetime.date(2022, 10, 7)

In [19]:
df[df['time_post'] == "2022-10-07"] 

,index,time_post,title_post,link,contents,words,top,count,top_title,count_title,stop,count_stop
14,155259,2022-10-07,Как мобилизованному сохранить статус ИП и прио...,https://buh.ru/news/uchet_nalogi/155259/,"Эксперты ФНС разъяснили, как мобилизованный ИП...","[эксперт, фнс, разъяснили, как, мобилизова, пр...","[Налог, ФНС]",2,[Налог],1,[],0
7,155274,2022-10-07,ФНС установит для перевозчиков формат электрон...,https://buh.ru/news/uchet_nalogi/155274/,ФНС РФ разработала формат электронного договор...,"[фнс, разработа, формат, электрон, договор, фр...","[Утвержден, ФНС]",2,[],0,[],0
18,155251,2022-10-07,Мобилизованные граждане освобождены от уплаты ...,https://buh.ru/news/uchet_nalogi/155251/,"ФНС предупредила, что мобилизованные граждане ...","[фнс, предупредила, что, мобилизова, гражданин...","[Налог, ФНС]",2,[Налог],1,[],0
16,155253,2022-10-07,Можно ли перейти с УСН или ЕСХН на автоматизир...,https://buh.ru/news/uchet_nalogi/155253/,"Эксперты ФНС разъяснили, как налогоплательщику...","[эксперт, фнс, разъяснили, как, налогоплательщ...",[ФНС],1,[],0,[],0
19,155249,2022-10-07,"ФНС изменит список стран, с которыми осуществл...",https://buh.ru/news/uchet_nalogi/155249/,ФНС подготовила новый перечень государств и те...,"[фнс, подготов, нов, перечен, государств, терр...",[ФНС],1,[],0,[],0
4,155277,2022-10-07,Вправе ли работодатель отправлять форму СТД-Р ...,https://buh.ru/news/uchet_nalogi/155277/,"Эксперты Роструда разъяснили, вправе ли работо...","[эксперт, роструд, разъяснили, вправ, работода...",[],0,[],0,[],0
11,155266,2022-10-07,Вступил в силу закон о кредитных каникулах для...,https://buh.ru/news/uchet_nalogi/155266/,С 7 октября 2022 года вступил в силу Федеральн...,"[октябр, 2022, год, вступ, сил, федеральн, зак...",[],0,[],0,[],0
10,155271,2022-10-07,В ТК РФ внесли дополнительные трудовые гаранти...,https://buh.ru/news/uchet_nalogi/155271/,"В Трудовой кодекс внесены новые положения, кот...","[трудов, кодекс, внест, нов, положения, котор,...",[],0,[],0,[],0
2,155279,2022-10-07,Работодателей обязали сдавать СЗВ-ТД при приос...,https://buh.ru/news/uchet_nalogi/155279/,С 7 октября 2022 года вступили в силу поправки...,"[октябр, 2022, год, вступ, сил, поправк, закон...",[],0,[],0,[],0


In [20]:
# Выводим вчерашние ранжированные новости в количестве 3-х штук
news = df[df['time_post'] == "2022-10-07"]['link'][:3]
news

14    https://buh.ru/news/uchet_nalogi/155259/
7     https://buh.ru/news/uchet_nalogi/155274/
18    https://buh.ru/news/uchet_nalogi/155251/
Name: link, dtype: object